# Notebook 2: Feature Engineering for Marketing Mix Modeling
## Creating Features that Capture Marketing Dynamics

**Learning Objectives:**
- Master adstock transformation for marketing carryover effects
- Implement Beta-Gamma features for saturation modeling
- Create price elasticity and promotional features
- Build seasonal and trend components

---

## The Science of Marketing Features

Marketing doesn't work instantly - it has carryover effects and saturation points. This notebook transforms raw data into features that capture these real-world dynamics.

In [ ]:
# Import libraries and load data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Load unified dataset from Notebook 1
data = pd.read_csv('data/processed/unified_mmm_data.csv')
data['Date'] = pd.to_datetime(data['Date'])

# Clean column names
data.columns = data.columns.str.strip().str.replace(' ', '_')

# Identify marketing channels
marketing_channels = ['TV', 'Sponsorship', 'Content_Marketing', 'Digital',
                      'SEM', 'Affiliates', 'Online_marketing', 'Radio']

print(f"Loaded dataset: {data.shape}")
print(f"Marketing channels: {len(marketing_channels)}")

## Part 1: Adstock Transformation

Marketing effects carry over time. A TV ad today influences purchases tomorrow and beyond. The adstock transformation models this decay.

In [ ]:
def apply_adstock(x, decay_rate=0.7, max_lag=3):
    '''Apply adstock transformation for carryover effects'''
    x = np.array(x, dtype=np.float64)
    adstocked = np.zeros_like(x)

    for lag in range(max_lag + 1):
        decay = decay_rate ** lag
        if lag == 0:
            adstocked += decay * x
        else:
            shifted = np.zeros_like(x)
            shifted[lag:] = x[:-lag]
            adstocked += decay * shifted
    return adstocked

# Channel-specific decay rates
decay_rates = {
    'TV': 0.8,            # Brand building - slow decay
    'Sponsorship': 0.75,  # Brand building
    'Content_Marketing': 0.6,
    'Digital': 0.4,       # Performance - fast decay
    'SEM': 0.3,          # Performance - very fast
    'Affiliates': 0.3,
    'Online_marketing': 0.4,
    'Radio': 0.5
}

# Apply adstock to all channels
print("Creating adstock features...")
data_sorted = data.sort_values(['product_category', 'Date'])

for channel in marketing_channels:
    if channel in data_sorted.columns:
        decay = decay_rates.get(channel, 0.5)
        adstock_col = f"{channel}_adstock"
        data_sorted[adstock_col] = data_sorted.groupby('product_category')[channel].transform(
            lambda x: apply_adstock(x.values, decay_rate=decay)
        )
        print(f"  Created {adstock_col} (decay={decay})")

data = data_sorted

## Part 2: Beta-Gamma Feature Preparation

Beta-Gamma transformation captures marketing saturation - the diminishing returns as spend increases.

In [ ]:
# Create features for Beta-Gamma transformation
beta_gamma_features = []

# Add all adstock features
adstock_features = [col for col in data.columns if '_adstock' in col]
beta_gamma_features.extend(adstock_features)

# Create log-transformed versions
for channel in marketing_channels:
    if channel in data.columns:
        # Log of raw spend
        log_col = f"{channel}_log"
        data[log_col] = np.log1p(data[channel])
        beta_gamma_features.append(log_col)

        # Log of adstock
        adstock_col = f"{channel}_adstock"
        if adstock_col in data.columns:
            log_adstock_col = f"{channel}_adstock_log"
            data[log_adstock_col] = np.log1p(data[adstock_col])
            beta_gamma_features.append(log_adstock_col)

print(f"\nCreated {len(beta_gamma_features)} Beta-Gamma features")
print(f"These features will model marketing saturation curves!")

In [ ]:
# Create price and promotional features
data['Discount_Pct'] = ((data['Avg_MRP'] - data['Avg_Price']) / (data['Avg_MRP'] + 0.01)) * 100
data['Price_Index'] = data.groupby('product_category')['Avg_Price'].transform(lambda x: x / x.mean())

# Monotonic constraints for business logic
monotonic_features = {
    'Avg_Price': 'negative',      # Higher price -> Lower sales
    'Avg_MRP': 'negative',
    'Discount_Pct': 'positive',   # Higher discount -> Higher sales
}

print("Price features created with monotonic constraints")

In [ ]:
# Create temporal features
data['Month'] = data['Date'].dt.month
data['Quarter'] = data['Date'].dt.quarter
data['DayOfWeek'] = data['Date'].dt.dayofweek

# Cyclical encoding for seasonality
data['Month_sin'] = np.sin(2 * np.pi * data['Month'] / 12)
data['Month_cos'] = np.cos(2 * np.pi * data['Month'] / 12)

print("Temporal features created")

In [ ]:
# Feature summary
print("="*60)
print("FEATURE ENGINEERING COMPLETE")
print("="*60)
print(f"Total features: {len(data.columns)}")
print(f"Beta-Gamma features: {len(beta_gamma_features)}")
print(f"Monotonic constraints: {len(monotonic_features)}")

# Save processed data
data.to_csv('data/processed/mmm_data_with_features.csv', index=False)
print("\nData saved to 'data/processed/mmm_data_with_features.csv'")